In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define a simple neural network class
class SimpleNet(nn.Module):
    def __init__(self, symmetric_weights=False):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(2, 2, bias=False)
        
        # If symmetric weights are desired, manually set them
        if symmetric_weights:
            with torch.no_grad():
                self.fc1.weight[0].fill_(0.5)
                self.fc1.weight[1].fill_(0.4)

    def forward(self, x):
        x = self.fc1(x)
        return x

# Create two networks, one with symmetric and one with asymmetric weights
net_symmetric = SimpleNet(symmetric_weights=True)
net_asymmetric = SimpleNet(symmetric_weights=False)

# Print initial weights for comparison
print("Symmetric Weights:\n", net_symmetric.fc1.weight)
print("\nAsymmetric Weights:\n", net_asymmetric.fc1.weight)

# Define a simple input
input = torch.tensor([1.0, 2.0])
target = torch.tensor([2.0, 1.0])

# Compare the outputs
output_symmetric = net_symmetric(input)
output_asymmetric = net_asymmetric(input)

print("\nOutput with Symmetric Weights:\n", output_symmetric)
print("\nOutput with Asymmetric Weights:\n", output_asymmetric)


optimizer = torch.optim.AdamW(list(net_symmetric.parameters()) + list(net_asymmetric.parameters()), lr=1e-3)

loss_fn = nn.MSELoss()

epochs = 10000
for epoch in range(epochs):
    
    output_symmetric = net_symmetric(input)
    output_asymmetric = net_asymmetric(input)

    loss_symmetric = loss_fn(output_symmetric, target)
    loss_asymmetric = loss_fn(output_asymmetric, target)

    if epoch == 1:
        print("\nOutput with Symmetric Weights:\n", output_symmetric)
        print("\nOutput with Asymmetric Weights:\n", output_asymmetric)

    optimizer.zero_grad()
    loss_symmetric.backward()
    loss_asymmetric.backward()
    optimizer.step()

# Compare the outputs
output_symmetric = net_symmetric(input)
output_asymmetric = net_asymmetric(input)

# Print initial weights for comparison
print("Symmetric Weights:\n", net_symmetric.fc1.weight)
print("\nAsymmetric Weights:\n", net_asymmetric.fc1.weight)

print("\nOutput with Symmetric Weights:\n", output_symmetric)
print("\nOutput with Asymmetric Weights:\n", output_asymmetric)

Symmetric Weights:
 Parameter containing:
tensor([[0.5000, 0.5000],
        [0.4000, 0.4000]], requires_grad=True)

Asymmetric Weights:
 Parameter containing:
tensor([[-0.2730, -0.5135],
        [ 0.5311,  0.0217]], requires_grad=True)

Output with Symmetric Weights:
 tensor([1.5000, 1.2000], grad_fn=<SqueezeBackward4>)

Output with Asymmetric Weights:
 tensor([-1.2999,  0.5745], grad_fn=<SqueezeBackward4>)

Output with Symmetric Weights:
 tensor([1.5030, 1.1970], grad_fn=<SqueezeBackward4>)

Output with Asymmetric Weights:
 tensor([-1.2969,  0.5775], grad_fn=<SqueezeBackward4>)
Symmetric Weights:
 Parameter containing:
tensor([[0.6667, 0.6667],
        [0.3333, 0.3333]], requires_grad=True)

Asymmetric Weights:
 Parameter containing:
tensor([[0.8116, 0.5941],
        [0.6406, 0.1797]], requires_grad=True)

Output with Symmetric Weights:
 tensor([2.0000, 1.0000], grad_fn=<SqueezeBackward4>)

Output with Asymmetric Weights:
 tensor([1.9999, 1.0000], grad_fn=<SqueezeBackward4>)


In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define a more complex neural network class
class ComplexNet(nn.Module):
    def __init__(self, symmetric_weights=False):
        super(ComplexNet, self).__init__()
        self.fc1 = nn.Linear(2, 2, bias=False)
        self.fc2 = nn.Linear(2, 2, bias=False)  # Additional layer

        # If symmetric weights are desired, manually set them
        if symmetric_weights:
            with torch.no_grad():
                self.fc1.weight[0].fill_(0.5)
                self.fc1.weight[1].fill_(0.4)
                self.fc2.weight.fill_(0.5)  # Set weights for the second layer as well

    def forward(self, x):
        x = F.relu(self.fc1(x))  # Adding non-linear activation function
        x = self.fc2(x)
        return x

# Create two networks, one with symmetric and one with asymmetric weights
net_symmetric = ComplexNet(symmetric_weights=True)
net_asymmetric = ComplexNet(symmetric_weights=False)

# Print initial weights for comparison
print("Symmetric Weights:\n", net_symmetric.fc1.weight)
print("\nAsymmetric Weights:\n", net_asymmetric.fc1.weight)

# Define a simple input and target
input = torch.tensor([1.0, 2.0])
target = torch.tensor([2.0, 1.0])

# Initialize optimizer
optimizer = torch.optim.AdamW(list(net_symmetric.parameters()) + list(net_asymmetric.parameters()), lr=1e-3)
loss_fn = nn.MSELoss()

epochs = 10000
for epoch in range(epochs):
    optimizer.zero_grad()

    output_symmetric = net_symmetric(input)
    output_asymmetric = net_asymmetric(input)

    loss_symmetric = loss_fn(output_symmetric, target)
    loss_asymmetric = loss_fn(output_asymmetric, target)

    loss_symmetric.backward()
    loss_asymmetric.backward()
    optimizer.step()

    if epoch in [1, epochs-1]:
        print(f"\nEpoch {epoch}")
        print("Output with Symmetric Weights:", output_symmetric)
        print("Output with Asymmetric Weights:", output_asymmetric)

# Final weights and outputs
print("\nFinal Symmetric Weights:\n", net_symmetric.fc1.weight)
print("\nFinal Asymmetric Weights:\n", net_asymmetric.fc1.weight)

print("\nFinal Output with Symmetric Weights:\n", net_symmetric(input))
print("\nFinal Output with Asymmetric Weights:\n", net_asymmetric(input))


Symmetric Weights:
 Parameter containing:
tensor([[0.5000, 0.5000],
        [0.4000, 0.4000]], requires_grad=True)

Asymmetric Weights:
 Parameter containing:
tensor([[0.1954, 0.0780],
        [0.3082, 0.6438]], requires_grad=True)

Epoch 1
Output with Symmetric Weights: tensor([1.3557, 1.3503], grad_fn=<SqueezeBackward4>)
Output with Asymmetric Weights: tensor([0.8707, 0.5941], grad_fn=<SqueezeBackward4>)

Epoch 9999
Output with Symmetric Weights: tensor([2.0000, 1.0000], grad_fn=<SqueezeBackward4>)
Output with Asymmetric Weights: tensor([2.0000, 1.0000], grad_fn=<SqueezeBackward4>)

Final Symmetric Weights:
 Parameter containing:
tensor([[0.6063, 0.6063],
        [0.5165, 0.5165]], requires_grad=True)

Final Asymmetric Weights:
 Parameter containing:
tensor([[0.4959, 0.3897],
        [0.5046, 0.8083]], requires_grad=True)

Final Output with Symmetric Weights:
 tensor([2.0000, 1.0000], grad_fn=<SqueezeBackward4>)

Final Output with Asymmetric Weights:
 tensor([2.0000, 1.0000], grad_fn

In [68]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define a neural network class with non-linear activation
class XORNet(nn.Module):
    def __init__(self, symmetric_weights=False):
        super(XORNet, self).__init__()
        self.fc1 = nn.Linear(2, 1, bias=False)

        # If symmetric weights are desired, manually set them
        if symmetric_weights:
            with torch.no_grad():
                self.fc1.weight.fill_(0.5)

    def forward(self, x):
        x = self.fc1(x)
        return x

# XOR input and output pairs
inputs = torch.tensor([[0., 0.], [0., 1.], [1., 0.], [1., 1.]])
targets = torch.tensor([[0.], [1.], [1.], [0.]])

# Create two networks
net_symmetric = XORNet(symmetric_weights=True)
net_asymmetric = XORNet(symmetric_weights=False)

# Initialize optimizer
optimizer_symmetric = torch.optim.Adam(net_symmetric.parameters(), lr=0.01)
optimizer_asymmetric = torch.optim.Adam(net_asymmetric.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

# Training loop
epochs = 1000
for epoch in range(epochs):
    for input, target in zip(inputs, targets):
        optimizer_symmetric.zero_grad()
        optimizer_asymmetric.zero_grad()

        output_symmetric = net_symmetric(input)
        output_asymmetric = net_asymmetric(input)

        loss_symmetric = loss_fn(output_symmetric, target)
        loss_asymmetric = loss_fn(output_asymmetric, target)

        loss_symmetric.backward()
        loss_asymmetric.backward()

        optimizer_symmetric.step()
        optimizer_asymmetric.step()

# Testing the trained networks
for input, target in zip(inputs, targets):
    print(f"Input: {input}")
    output_symmetric = net_symmetric(input)
    output_asymmetric = net_asymmetric(input)
    print(f"Symmetric Output: {output_symmetric}, Asymmetric Output: {output_asymmetric}")


Input: tensor([0., 0.])
Symmetric Output: tensor([0.], grad_fn=<SqueezeBackward4>), Asymmetric Output: tensor([0.], grad_fn=<SqueezeBackward4>)
Input: tensor([0., 1.])
Symmetric Output: tensor([0.3332], grad_fn=<SqueezeBackward4>), Asymmetric Output: tensor([0.3332], grad_fn=<SqueezeBackward4>)
Input: tensor([1., 0.])
Symmetric Output: tensor([0.3332], grad_fn=<SqueezeBackward4>), Asymmetric Output: tensor([0.3332], grad_fn=<SqueezeBackward4>)
Input: tensor([1., 1.])
Symmetric Output: tensor([0.6663], grad_fn=<SqueezeBackward4>), Asymmetric Output: tensor([0.6663], grad_fn=<SqueezeBackward4>)
